In [ ]:
# Step 1: Install ultralytics
!pip install ultralytics

# Step 2: Import required libraries
import os
import shutil
import torch
from ultralytics import YOLO
import logging
from datetime import datetime

# Step 3: Allowlist DetectionModel to avoid UnpicklingError
from ultralytics.nn.tasks import DetectionModel
torch.serialization.add_safe_globals([DetectionModel])

# Step 4: Set up logging to track progress
log_file = '/kaggle/working/plastic_detection/training_log.txt'
os.makedirs(os.path.dirname(log_file), exist_ok=True)

logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

logging.info("Starting YOLOv8 training process on Kaggle")

# Step 5: Copy dataset from read-only /kaggle/input to writable /kaggle/working
input_dataset_path = '/kaggle/input/plastic-type-dataset/Plastic-Recyclable-Detection-Updated'  # Updated path with correct capitalization
working_dataset_path = '/kaggle/working/Plastic-Recyclable-Detection-Updated'

# Check if the input dataset path exists
if not os.path.exists(input_dataset_path):
    error_msg = f"Dataset not found at {input_dataset_path}. Available datasets: {os.listdir('/kaggle/input')}"
    logging.error(error_msg)
    raise FileNotFoundError(error_msg)

# Check the contents of the dataset
logging.info(f"Contents of {input_dataset_path}: {os.listdir(input_dataset_path)}")

# Copy dataset to writable directory
if not os.path.exists(working_dataset_path):
    shutil.copytree(input_dataset_path, working_dataset_path)
    logging.info(f"Copied dataset from {input_dataset_path} to {working_dataset_path}")
else:
    logging.info(f"Dataset already exists at {working_dataset_path}")

# Step 6: Update data.yaml to point to the new paths
data_yaml_path = os.path.join(working_dataset_path, 'data.yaml')

# Verify the data.yaml file exists
if not os.path.exists(data_yaml_path):
    error_msg = f"data.yaml not found at {data_yaml_path}. Available files: {os.listdir(working_dataset_path)}"
    logging.error(error_msg)
    raise FileNotFoundError(error_msg)

# Read the original data.yaml and update paths
with open(data_yaml_path, 'r') as file:
    data_yaml_content = file.read()

# Replace old paths with new paths
data_yaml_content = data_yaml_content.replace('/content/drive/MyDrive/yolo1/Plastic-Recyclable-Detection-Updated', working_dataset_path)

# Write the updated data.yaml
with open(data_yaml_path, 'w') as file:
    file.write(data_yaml_content)

logging.info(f"Updated data.yaml at {data_yaml_path}")

# Step 7: Determine device (GPU if available, CPU otherwise)
if torch.cuda.is_available():
    device = 'cuda:0'
    logging.info(f"GPU available, using device: {device}")
else:
    device = 'cpu'
    logging.info(f"No GPU available, using device: {device}")

# Step 8: Load the YOLOv8 model
try:
    model = YOLO('yolov8n.pt')
    logging.info("YOLOv8 model loaded successfully")
except Exception as e:
    logging.error(f"Failed to load YOLOv8 model: {str(e)}")
    raise

# Step 9: Train the model with optimized settings
try:
    batch_size = 16 if torch.cuda.is_available() else 4
    results = model.train(
        data=data_yaml_path,
        epochs=10,  # Reduced for faster training
        imgsz=416,  # Reduced image size for faster training
        batch=batch_size,
        name='plastic_detection',
        project='/kaggle/working/plastic_detection',
        device=device,
        patience=10,
        save=True,
        save_period=2,
        exist_ok=True
    )
    logging.info("Training completed successfully")
except Exception as e:
    logging.error(f"Training failed: {str(e)}")
    raise

# Step 10: Save the trained model
model_path = '/kaggle/working/plastic_detection/weights/best.pt'
if os.path.exists(model_path):
    logging.info(f"Trained model saved at {model_path}")
else:
    logging.warning("Training may have failed; best.pt not found")

# Step 11: Test inference
try:
    model = YOLO(model_path)
    test_image_path = os.path.join(working_dataset_path, 'test', 'images', os.listdir(os.path.join(working_dataset_path, 'test', 'images'))[0])
    results = model.predict(
        source=test_image_path,
        conf=0.25,
        iou=0.45,
        save=True,
        save_txt=True,
        project='/kaggle/working/plastic_detection',
        name='test_inference',
        exist_ok=True
    )
    logging.info(f"Inference result saved at /kaggle/working/plastic_detection/test_inference/")
except Exception as e:
    logging.error(f"Inference failed: {str(e)}")
    raise

logging.info("Script completed successfully")